In [1]:
import os
import json
import pickle
import requests
import numpy as np
from pprint import pprint
import pandas as pd
import json
import pickle
from pprint import pprint

In [2]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

# 1. Match Tail Entities Synsets to ConceptNet Index

Remember that the original prep step already matched "word.noun" to "c/en/word/n" formats so we only have to work on the tail labels

## 1.1 Load Excluded Facts DataFrame (preprocessed) and add synsets

In [3]:
df_excluded = pd.read_csv('df_excluded_240_20211010.csv')
print(df_excluded.shape)
df_excluded.head()

(240, 3)


,entity_head_vgg,predicate,entity_tail_wikidata
0,chair,/r/IsA,academic_department
1,umbrella,/r/IsA,accessory
2,helmet,/r/PartOf,achievement
3,handle,/r/IsA,address
4,tower,/r/IsA,amusement_ride


In [4]:
wikidata_labels_matched = [
    'academic_department.n.01',
    'accessory.n.01',
    'accomplishment.n.01',
    'address.n.01',
    'ride.n.02',
    'structure.n.04',
    'structure.n.04',
    'artifact.n.01',
    'component.n.03',
    'component.n.03',
    'component.n.03',
    'structure.n.01',
    'structure.n.01',
    'structure.n.01',
    'structure.n.01',
    'material.n.04',
    'article.n.01',
    'asset.n.01',
    'astrology.n.01',
    'auditorium.n.01',
    'badminton.n.01',
    'baggage.n.01',
    'bat.n.05',
    'bat.n.05',
    'bedclothes.n.01',
    'bell_tower.n.01',
    'berlin.n.03',
    'bin.n.01',
    'blade.n.09',
    'blend.n.01',
    'body_armor.n.01',
    'body_part.n.01',
    'body_part.n.01',
    'body_part.n.01',
    'body_part.n.01',
    'body_part.n.01',
    'body_part.n.01',
    'bonnet.n.01',
    'box.n.01', # circular
    'brake.n.01',
    'branch.n.01', # circular
    'branch.n.01',
    'branch.n.01',
    'brig.n.01',
    'building.n.01',
    'building.n.01',
    'building.n.01',
    'building_material.n.01',
    'capital.n.03', # incorrect fact?
    'glass.n.01',
    'cardboard.n.01',
    'cartridge.n.01',
    'caldron.n.01',
    'character.n.08',
    'charge.n.03', # not facts?
    'charge.n.03', # not facts?
    'charge.n.03', # not facts?
    'hair.n.01',
    'church.n.02',
    'citrus.n.01',
    'clock_tower.n.01',
    'coat.n.01',
    'cockpit.n.01',
    'block.n.06',
#     'collar.n.08', # 88 fence /r/Synonym collar
    'color.n.01',
#     'color', # 241 coat /r/IsA color
#     'combination', # 167 hand /r/IsA combination
    'protocol.n.01',
    'coping_saw.n.01',
    'design.n.04',
    'headquarters.n.01',
    'cotton.n.01',
#     'counter', # 244 counter /r/DistinctFrom counter
    'work.n.02',
#     'cricket_whites', # shirt /r/PartOf cricket_whites
    'mafia.n.01',
    'crowd.n.01',
    'crown.n.07',
#     'cushion.n.03', # 172 pillow /r/DistinctFrom cushion
    'cushion.n.03',
    'darts.n.01',
    'data_structure.n.01',
    'department.n.01',
    'derivative.n.02',
    'development.n.01',
    'device.n.01',
#     'documentary_film', # 18 truck /r/HasContext documentary_film
    'land.n.04',
    'eighth_note.n.01',
    'storehouse.n.01',
#     'estate', # 200 branch /r/IsA estate
    'executive_branch.n.01',
    'sports_equipment.n.01',
#     'eye_color', # 238 orange /r/IsA eye_color because no results. orange is color is duplicate
    'surname.n.01',
    'fin.n.05',
    'flax.n.01',
    'design.n.04',
    'footwear.n.01',
    'footwear.n.01',
#     'fork', # 124 fork /r/DistinctFrom fork
    'geological_formation.n.01',
#     'fortification', # 49 cabinet /r/PartOf fortification
    'foundation.n.03',
    'fuselage.n.01',
    'game_equipment.n.01',
    'game_equipment.n.01',
    'man.n.10',
    'gesture.n.01',
    'barrel.n.01',
    'signal.n.01',
#     'head', # 91 head /r/IsA head
    'dwelling.n.01',
    'hood.n.08',
    'buggy.n.01',
    'house.n.01',
#     'house', 30 house /r/IsA house
    'human_body.n.01',
    'human_body.n.01',
    'human_body.n.01',
    'ice_hockey.n.01',
#     'incentive_program', # 63 skateboard /r/IsA incentive_program
    'person.n.01',
    'affinity.n.04',
#     'kite', # 87 kite /r/HasContext kite
    'cap.n.01',
    'lacrosse.n.01',
    'stick.n.06',
# starts here#     'letter', # 223 letter /r/DistinctFrom letter
    'light.n.02',
    'literary_composition.n.01',
    'local_road.n.01',
    'log.n.01',
    'lumber.n.01',
    'marital_status.n.01',
    'marker.n.02',
#     'medical_term', # 138 wing /r/IsA medical_term
#     'member', # 1 bed /r/PartOf member
    'meme.n.01',
    'metallic_element.n.01',
    'martial_art.n.01',
    'martial_art.n.01',
    'conveyance.n.03',
    'conveyance.n.03',
    'motif.n.01',
    'motif.n.01',
    'musical_notation.n.01',
    'work.n.02',
#     'nasal_hair', # 275 hair /r/PartOf nasal_hair - cycle
#     'natural_person', # 297 person /r/Synonym natural_person
    'term.n.01',
    'node.n.08',
    'title.n.09',
#     'note', # 214 flag /r/PartOf note - ???
    'nutrient.n.02',
    'operating_system.n.01',
    'orange.n.03',
    'orange.n.02',
#     'orange_chicken', # 231 orange /r/PartOf orange_chicken - doesn't exist
#     'orange_jam', # 230 orange /r/PartOf orange_jam - nonexistent
    'orange_juice.n.01',
    'organ.n.01',
    'pavement.n.01',
    'establishment.n.04',
    'establishment.n.04',
    'railway.n.01',
    'personal_computer.n.01',
    'life.n.03',
    'phrase.n.01',
    'pin.n.09',
#     'plane', # 162 plane /r/IsA plane - cycle
#     'plane', # 116 plate /r/HasProperty plane - ???
    'plant_part.n.01',
    'plant_part.n.01',
    'plant_part.n.01',
    'plant_part.n.01',
    'plant_part.n.01',
    'stalk.n.02',
    'plastic.n.01',
    'plastic.n.01',
    'plumbing_fixture.n.01',
    'portable_computer.n.01',
    'placement.n.03',
    'position.n.06',
    'placement.n.03',
    'protection.n.01',
    'protective_covering.n.01',
    'pseudonym.n.01',
#     'racket', # 258 racket /r/IsA racket - cycle
#     'racket', # 257 racket /r/DistinctFrom racket
#     'radiator', # 278 jacket /r/DistinctFrom radiator - nonexistent
    'switch.n.04',
    'reading.n.01',
    'building.n.01',
    'consequence.n.01',
    'function.n.03',
    'path.n.03',
    'sailing.n.02',
    'sanitation.n.01',
    'score.n.03',
    'script.n.03',
#     'seat', # 39 seat /r/DistinctFrom seat - cycle
    'seat.n.03',
#     'secrecy', # 165 hand /r/HasProperty secrecy - ???
    'sequence.n.01',
    'plate_glass.n.01',
    'shoe.n.01',
    'size.n.02',
    'sign.n.02',
#     'sign', # 5 sign /r/Synonym sign -- ??? 
    'sign_language.n.01',
    'silver.n.01',
    'sixteenth_note.n.01',
    'skateboard.n.01',
    'skateboarding.n.01',
    'skateboarding.n.01',
    'skiing.n.01',
    'skiing.n.01',
    'skiing.n.01',
#     'social_consciousness', # 289 person /r/Antonym social_consciousness - nonexistent
    'social_organization.n.01',
    'source.n.07',
    'space.n.03',
    'violin.n.01',
    'speech.n.02',
    'sports_equipment.n.01',
    'sports_equipment.n.01',
    'sports_equipment.n.01',
    'sports_equipment.n.01',
    'paint.n.01',
    'state_government.n.01',
    'steel.n.01',
    'rock.n.01',
    'rock.n.02',
    'storage.n.01',
    'street.n.01',
#     'street', # 105 street /r/IsA street - cycle
    'furniture.n.01',
    'component.n.01', # not really sure.
    'national.n.01',
    'subsidiary_company.n.01',
#     'supporter', # 120 woman /r/IsA supporter --- ?
#     'supporter', # 71 man /r/IsA supporter --- ?
    'surface.n.02',
    'surveying.n.01',
    'syllable.n.01',
    'symbol.n.01',
    'symbol.n.02',
    'system_call.n.01',
    'table.n.02',
    'tactic.n.01',
    'team_sport.n.01',
    'documentary.n.01',
    'toilet.n.01',
    'tool.n.01',
    'tool.n.01',
    'creature.n.03',
    'top.n.10',
    'torture.n.05',
#     'torture_instrument', # 157 box /r/IsA torture_instrument - nonexistent
#     'torture_instrument', # 28 boot /r/IsA torture_instrument - nonexistent
#     'tower', # 25 tower /r/HasContext tower - cycle
#     'track', # 107 track /r/Synonym track
    'pylon.n.02',
    'trophy.n.02',
#     'trunk', # 302 trunk /r/DistinctFrom trunk
    'trunk.n.01', 
    'unit_of_measurement.n.01',
#     'value', # 166 hand /r/HasProperty value - ???
    'ventilation.n.01',
    'venue.n.01',
    'vessel.n.02',
#     'victoria', # 286 bench /r/PartOf victoria - ???
    'artist.n.01',
#     'vocal_sound', # 249 phone /r/IsA vocal_sound - cycle
    'voting_booth.n.01',
    'lounge.n.02',
    'lounge.n.02',
    'color.n.01',
    'color.n.01',
    'exercise.n.01',
#     'window_treatment', # 15 curtain /r/PartOf window_treatment - nonexistent
#     'window_treatment', # 255 window /r/PartOf window_treatment - nonexistent
#     'wing', # 141 wing /r/IsA wing - cycle
    'yellow.n.01'
]

In [5]:
df_excluded['synset'] = wikidata_labels_matched
label2synset_wikidata = {l: s for l, s in zip(list(df_excluded['entity_tail_wikidata']), list(df_excluded['synset']))}

In [6]:
df_excluded.head()

,entity_head_vgg,predicate,entity_tail_wikidata,synset
0,chair,/r/IsA,academic_department,academic_department.n.01
1,umbrella,/r/IsA,accessory,accessory.n.01
2,helmet,/r/PartOf,achievement,accomplishment.n.01
3,handle,/r/IsA,address,address.n.01
4,tower,/r/IsA,amusement_ride,ride.n.02


In [7]:
# def get_conceptnet_id(synset):
#     subq = synset.split('.')
#     if len(subq) >= 2:
#         subq = subq[0] + '/' + subq[1]
#     else:
#         subq = subq[0]
#     q = f"http://api.conceptnet.io/c/en/{subq}?offset=0&limit=1000"
#     obj = requests.get(q).json()    
#     if len(obj['edges']) == 0:
#         subq = subq[0].replace(' ', '_')
#         q = f"http://api.conceptnet.io/c/en/{subq}?offset=0&limit=1000"
#         obj = requests.get(q).json()
#     if len(obj['edges']) == 0:
#         subq = subq[0].replace(' ', '_')
#         q = f"http://api.conceptnet.io/c/en/{subq}?offset=0&limit=1000"
#         obj = requests.get(q).json()
#     return obj['@id']

# def get_conceptnet_id_with_synset_dict(label, synset_dict):
#     subq = synset_dict[label].split('.')
#     if len(subq) >= 2:
#         subq = subq[0] + '/' + subq[1]
#     else:
#         subq = label
#     q = f"http://api.conceptnet.io/c/en/{subq}?offset=0&limit=1000"
#     obj = requests.get(q).json()    
#     if len(obj['edges']) == 0:
#         subq = label.replace(' ', '_')
#         q = f"http://api.conceptnet.io/c/en/{subq}?offset=0&limit=1000"
#         obj = requests.get(q).json()
#     if len(obj['edges']) == 0:
#         subq = label.replace(' ', '_')
#         q = f"http://api.conceptnet.io/c/en/{subq}?offset=0&limit=1000"
#         obj = requests.get(q).json()
#     return obj['@id']

# 1.? Get ConceptNet IDs for New Entity Tails

In [131]:
labels_wikidata_tail

0      academic_department
1                accessory
2              achievement
3                  address
4           amusement_ride
              ...         
235           waiting_room
236              web_color
237              web_color
238        weight_training
239                 yellow
Name: entity_tail_wikidata, Length: 240, dtype: object

In [133]:
len(set(labels_wikidata_tail))

196

In [8]:
# Already run and saved
labels_wikidata_tail = df_excluded['entity_tail_wikidata']
# label2id_wikidata = {}
# for l in labels_wikidata_tail:
#     subq = label2synset_wikidata[l].split('.')
#     if len(subq) >= 2:
#         subq = subq[0] + '/' + subq[1]
#     else:
#         subq = l
#     q = f"http://api.conceptnet.io/c/en/{subq}?offset=0&limit=1000"
#     obj = requests.get(q).json()    
#     if len(obj['edges']) == 0:
#         subq = l.replace(' ', '_')
#         q = f"http://api.conceptnet.io/c/en/{subq}?offset=0&limit=1000"
#         obj = requests.get(q).json()
#     if len(obj['edges']) == 0:
#         subq = l.replace(' ', '_')
#         q = f"http://api.conceptnet.io/c/en/{subq}?offset=0&limit=1000"
#         obj = requests.get(q).json()
#     label2id_wikidata[l] = obj['@id']

In [94]:
# save_obj(label2id_wikidata, 'label2id_wikidata_196_20211018')
label2id_wikidata = load_obj('label2id_wikidata_196_20211018')

In [10]:
# Why is len(label2id_wikidata) only 196 but len(labels_wikidata_tail) is 240? Because there are 196 unique tail labels. So multiple facts share a tail label
len(labels_wikidata_tail.unique()) # 196
len(label2id_wikidata) # 196
len(labels_wikidata_tail) # 240

240

In [11]:
# def synset_to_conceptnet_id(synset):
#     synset_components = synset.split('.')
#     conceptnet_id = ''
#     if len(synset_components) >= 2:
#         conceptnet_id = synset_components[0] + '/' + synset_components[1]
#     else:
#         conceptnet_id = synset
#     return f"/c/en/{conceptnet_id}"

# print(synset_to_conceptnet_id('academic_department.n.01')) # /c/en/academic_department/n

df_excluded.loc[:, 'cn_id_tail'] = df_excluded.apply(lambda row: label2id_wikidata[row['entity_tail_wikidata']], axis=1)
## potentially wrong logic df_excluded.loc[:, 'cn_id_tail'] = df_excluded.apply(lambda row: get_conceptnet_id(row['synset']), axis=1)

In [12]:
df_excluded

,entity_head_vgg,predicate,entity_tail_wikidata,synset,cn_id_tail
0,chair,/r/IsA,academic_department,academic_department.n.01,/c/en/academic_department/n
1,umbrella,/r/IsA,accessory,accessory.n.01,/c/en/accessory/n
2,helmet,/r/PartOf,achievement,accomplishment.n.01,/c/en/accomplishment/n
3,handle,/r/IsA,address,address.n.01,/c/en/address/n
4,tower,/r/IsA,amusement_ride,ride.n.02,/c/en/ride/n
...,...,...,...,...,...
235,seat,/r/PartOf,waiting_room,lounge.n.02,/c/en/lounge/n
236,snow,/r/IsA,web_color,color.n.01,/c/en/color/n
237,orange,/r/IsA,web_color,color.n.01,/c/en/color/n
238,bench,/r/UsedFor,weight_training,exercise.n.01,/c/en/exercise/n


In [13]:
df_excluded

,entity_head_vgg,predicate,entity_tail_wikidata,synset,cn_id_tail
0,chair,/r/IsA,academic_department,academic_department.n.01,/c/en/academic_department/n
1,umbrella,/r/IsA,accessory,accessory.n.01,/c/en/accessory/n
2,helmet,/r/PartOf,achievement,accomplishment.n.01,/c/en/accomplishment/n
3,handle,/r/IsA,address,address.n.01,/c/en/address/n
4,tower,/r/IsA,amusement_ride,ride.n.02,/c/en/ride/n
...,...,...,...,...,...
235,seat,/r/PartOf,waiting_room,lounge.n.02,/c/en/lounge/n
236,snow,/r/IsA,web_color,color.n.01,/c/en/color/n
237,orange,/r/IsA,web_color,color.n.01,/c/en/color/n
238,bench,/r/UsedFor,weight_training,exercise.n.01,/c/en/exercise/n


## 1.2 Add ConceptNet Prefix for Head Entities from Original Preprocessed Data (Zareian prep_002.ipynb)

In [2]:
import json
with open('VG-SGG-dicts.json', 'r') as fin:
    scene_graph_meta = json.load(fin)
labels = list(scene_graph_meta['label_to_idx'].keys())
preds = list(scene_graph_meta['predicate_to_idx'].keys())
synsets = [
    'kite.n.03',
    'trouser.n.01',
    'bowl.n.01',
    'laptop.n.01',
    'paper.n.01',
    'motorcycle.n.01',
    'railing.n.01',
    'chair.n.01',
    'windshield.n.01',
    'tire.n.01',
    'cup.n.01',
    'bench.n.01',
    'tail.n.01',
    'bicycle.n.01',
    'board.n.03',
    'orange.n.01',
    'hat.n.01',
    'finger.n.01',
    'plate.n.04',
    'woman.n.01',
    'handle.n.01',
    'branch.n.02',
    'food.n.01',
    'bear.n.01',
    'vase.n.01',
    'vegetable.n.01',
    'giraffe.n.01',
    'desk.n.01',
    'lady.n.01',
    'towel.n.01',
    'glove.n.02',
    'bag.n.01',
    'nose.n.01',
    'rock.n.01',
    'guy.n.01',
    'shoe.n.01',
    'gym_shoe.n.01',
    'fence.n.01',
    'people.n.01',
    'house.n.01',
    'seat.n.01',
    'hair.n.01',
    'street.n.02',
    'roof.n.01',
    'racket.n.04',
    'logo.n.01',
    'female_child.n.01',
    'arm.n.01',
    'flower.n.01',
    'leaf.n.01',
    'clock.n.01',
    'hill.n.01',
    'bird.n.01',
    'umbrella.n.01',
    'leg.n.01',
    'screen.n.03',
    'man.n.01',
    'sink.n.01',
    'trunk.n.01',
    'post.n.04',
    'sidewalk.n.01',
    'box.n.01',
    'male_child.n.01',
    'cow.n.01',
    'skateboard.n.01',
    'airplane.n.01',
    'stand.n.04',
    'pillow.n.01',
    'ski.n.01',
    'wire.n.02',
    'toilet.n.02',
    'pot.n.01',
    'sign.n.02',
    'numeral.n.01',
    'pole.n.01',
    'table.n.02',
    'boat.n.01',
    'sheep.n.01',
    'horse.n.01',
    'eye.n.01',
    'sock.n.01',
    'window.n.01',
    'vehicle.n.01',
    'curtain.n.01',
    'child.n.01',
    'banana.n.01',
    'engine.n.01',
    'head.n.01',
    'door.n.01',
    'bus.n.01',
    'cabinet.n.01',
    'glass.n.02',
    'flag.n.01',
    'train.n.01',
    'child.n.01',
    'ear.n.01',
    'surfboard.n.01',
    'room.n.01',
    'player.n.01',
    'car.n.01',
    'cap.n.01',
    'tree.n.01',
    'bed.n.01',
    'cat.n.01',
    'coat.n.01',
    'skier.n.01',
    'zebra.n.01',
    'fork.n.01',
    'drawer.n.01',
    'airplane.n.01',
    'helmet.n.01',
    'shirt.n.01',
    'paw.n.01',
    'boot.n.01',
    'snow.n.02',
    'lamp.n.01',
    'book.n.02',
    'animal.n.01',
    'elephant.n.01',
    'tile.n.01',
    'necktie.n.01',
    'beach.n.01',
    'pizza.n.01',
    'wheel.n.01',
    'plant.n.02',
    'tower.n.01',
    'mountain.n.01',
    'path.n.04',
    'hand.n.01',
    'fruit.n.01',
    'mouth.n.01',
    'letter.n.02',
    'shelf.n.01',
    'wave.n.01',
    'man.n.01',
    'building.n.01',
    'short_pants.n.01',
    'neck.n.01',
    'telephone.n.01',
    'light.n.02',
    'counter.n.01',
    'dog.n.01',
    'face.n.01',
    'jacket.n.01',
    'person.n.01',
    'truck.n.01',
    'bottle.n.01',
    'basket.n.01',
    'jean.n.01',
    'wing.n.02',
]
label2synset = {l: s for l, s in zip(labels, synsets)}

In [3]:
preds

['and',
 'says',
 'belonging to',
 'over',
 'parked on',
 'growing on',
 'standing on',
 'made of',
 'attached to',
 'at',
 'in',
 'hanging from',
 'wears',
 'in front of',
 'from',
 'for',
 'watching',
 'lying on',
 'to',
 'behind',
 'flying in',
 'looking at',
 'on back of',
 'holding',
 'between',
 'laying on',
 'riding',
 'has',
 'across',
 'wearing',
 'walking on',
 'eating',
 'above',
 'part of',
 'walking in',
 'sitting on',
 'under',
 'covered in',
 'carrying',
 'using',
 'along',
 'with',
 'on',
 'covering',
 'of',
 'against',
 'playing',
 'near',
 'painted on',
 'mounted on']

In [1]:
# label2id = {}
# for l in labels:
#     label2id[l] = get_conceptnet_id(l)
# save_obj(label2id, 'label2id_150_zareian_code_20211017') # saved
label2id = load_obj('label2id_150_zareian_code_20211017')
# df_excluded.loc[:, 'cn_id_head'] = df_excluded.apply(lambda row: label2id[row['entity_head_vgg']], axis=1)

NameError: name 'load_obj' is not defined

# 2. Build Original Edges



In [16]:
# save_obj(pred2id_old, 'pred2id_old') # saved
pred2id = load_obj('pred2id_old')

In [17]:
cn_ids = {i for l, i in label2id.items()} | {i for l, i in pred2id.items()}

In [18]:
def add_to_edges(obj):
    for e in obj['edges']:
        e1 = findmatch(e['start']['@id'])
        e2 = findmatch(e['end']['@id'])
        r = e['rel']['@id']
        if e1 is None or e2 is None:
            continue
        if e1 == e2:
            continue
        edges.append((e1, e2, r))
        print('add_to_edges: added edge ({}, {}, {})'.format(e1, e2, r))

In [19]:
def findmatch(cnid):
    matched_ids = []
    for item in cn_ids:
        if cnid == item:
            matched_ids.append(item)
        elif cnid.startswith(item + '/'):            
            matched_ids.append(item)
    if len(matched_ids) > 1:
        raise
    return matched_ids[0] if len(matched_ids) > 0 else None


In [20]:
display(df_excluded['predicate'].value_counts())

/r/IsA             132
/r/PartOf           42
/r/UsedFor          15
/r/DistinctFrom     12
/r/MadeOf           11
/r/HasContext       10
/r/HasProperty       7
/r/Synonym           6
/r/DerivedFrom       2
/r/Antonym           2
/r/RelatedTo         1
Name: predicate, dtype: int64

In [21]:
edges = []
for l in labels:
    subq = label2id[l]
    q = f"http://api.conceptnet.io{subq}?offset=0&limit=1000"
    obj = requests.get(q).json()    
    add_to_edges(obj)
    while 'view' in obj and 'nextPage' in obj['view']:
        q = f"http://api.conceptnet.io{obj['view']['nextPage']}"
        obj = requests.get(q).json()
        add_to_edges(obj)
for l in preds:
    subq = pred2id[l]
    q = f"http://api.conceptnet.io{subq}?offset=0&limit=1000"
    obj = requests.get(q).json()    
    add_to_edges(obj)
    while 'view' in obj and 'nextPage' in obj['view']:
        q = f"http://api.conceptnet.io{obj['view']['nextPage']}"
        obj = requests.get(q).json()
        add_to_edges(obj)


add_to_edges: added edge (/c/en/kite/n, /c/en/bird/n, /r/IsA)
add_to_edges: added edge (/c/en/bowl/n, /c/en/food/n, /r/RelatedTo)
add_to_edges: added edge (/c/en/bowl/n, /c/en/hold/v, /r/UsedFor)
add_to_edges: added edge (/c/en/motorcycle/n, /c/en/ride/v, /r/UsedFor)
add_to_edges: added edge (/c/en/chair/n, /c/en/seat/n, /r/IsA)
add_to_edges: added edge (/c/en/leg/n, /c/en/chair/n, /r/PartOf)
add_to_edges: added edge (/c/en/chair/n, /c/en/people/n, /r/UsedFor)
add_to_edges: added edge (/c/en/chair/n, /c/en/seat/n, /r/IsA)
add_to_edges: added edge (/c/en/windshield/n, /c/en/airplane/n, /r/PartOf)
add_to_edges: added edge (/c/en/windshield/n, /c/en/screen/n, /r/IsA)
add_to_edges: added edge (/c/en/cup/n, /c/en/hold/v, /r/UsedFor)
add_to_edges: added edge (/c/en/bench/n, /c/en/seat/n, /r/IsA)
add_to_edges: added edge (/c/en/bench/n, /c/en/seat/n, /r/IsA)
add_to_edges: added edge (/c/en/bench/n, /c/en/people/n, /r/RelatedTo)
add_to_edges: added edge (/c/en/bench/n, /c/en/seat/n, /r/IsA)
ad

add_to_edges: added edge (/c/en/door/n, /c/en/room/n, /r/IsA)
add_to_edges: added edge (/c/en/bus/n, /c/en/car/n, /r/IsA)
add_to_edges: added edge (/c/en/bus/n, /c/en/transport/v, /r/UsedFor)
add_to_edges: added edge (/c/en/bus/n, /c/en/transport/v, /r/UsedFor)
add_to_edges: added edge (/c/en/window/n, /c/en/bus/n, /r/PartOf)
add_to_edges: added edge (/c/en/roof/n, /c/en/bus/n, /r/PartOf)
add_to_edges: added edge (/c/en/shelf/n, /c/en/cabinet/n, /r/PartOf)
add_to_edges: added edge (/c/en/cabinet/n, /c/en/box/n, /r/IsA)
add_to_edges: added edge (/c/en/cabinet/n, /c/en/box/n, /r/IsA)
add_to_edges: added edge (/c/en/glass/n, /c/en/hold/v, /r/UsedFor)
add_to_edges: added edge (/c/en/flag/n, /c/en/dog/n, /r/PartOf)
add_to_edges: added edge (/c/en/flag/n, /c/en/tail/n, /r/IsA)
add_to_edges: added edge (/c/en/child/n, /c/en/person/n, /r/IsA)
add_to_edges: added edge (/c/en/child/n, /c/en/animal/n, /r/IsA)
add_to_edges: added edge (/c/en/ear/n, /c/en/fruit/n, /r/IsA)
add_to_edges: added edge (

add_to_edges: added edge (/c/en/and, /c/en/to, /r/RelatedTo)
add_to_edges: added edge (/c/en/room/n, /c/en/to, /r/RelatedTo)
add_to_edges: added edge (/c/en/to, /c/en/at, /r/RelatedTo)
add_to_edges: added edge (/c/en/from, /c/en/to, /r/DistinctFrom)
add_to_edges: added edge (/c/en/fly/v, /c/en/transport/v, /r/MannerOf)
add_to_edges: added edge (/c/en/watch/v, /c/en/look/v, /r/Entails)
add_to_edges: added edge (/c/en/make/v, /c/en/look/v, /r/MannerOf)
add_to_edges: added edge (/c/en/leg/n, /c/en/hold/v, /r/UsedFor)
add_to_edges: added edge (/c/en/counter/n, /c/en/hold/v, /r/UsedFor)
add_to_edges: added edge (/c/en/hold/v, /c/en/cover/v, /r/MannerOf)
add_to_edges: added edge (/c/en/box/n, /c/en/hold/v, /r/UsedFor)
add_to_edges: added edge (/c/en/bag/n, /c/en/hold/v, /r/UsedFor)
add_to_edges: added edge (/c/en/basket/n, /c/en/hold/v, /r/UsedFor)
add_to_edges: added edge (/c/en/bowl/n, /c/en/hold/v, /r/UsedFor)
add_to_edges: added edge (/c/en/hold/v, /c/en/have/v, /r/Synonym)
add_to_edges:

In [22]:
edges_duplicate = edges.copy()
print(len(edges_duplicate))

# remove duplicates and convert to list of lists
edges_unique = list(set(edges_duplicate))
edges_unique = [list(a) for a in edges_unique]

323


In [23]:
print(len(edges_unique)) # 144, 3 fewer than the original 147

144


In [24]:
# save_obj(edges, 'edges_unique_144_20211017') # saved

In [25]:
# Load from Zareian's 147 edges
edges_zareian_enumerate = \
[(0, ['/c/en/windshield/n', '/c/en/screen/n', '/r/IsA']),
 (1, ['/c/en/cat/n', '/c/en/woman/n', '/r/IsA']),
 (2, ['/c/en/wing/n', '/c/en/airplane/n', '/r/PartOf']),
 (3, ['/c/en/room/n', '/c/en/to', '/r/RelatedTo']),
 (4, ['/c/en/bowl/n', '/c/en/hold/v', '/r/UsedFor']),
 (5, ['/c/en/play/v', '/c/en/use/v', '/r/MannerOf']),
 (6, ['/c/en/arm/n', '/c/en/branch/n', '/r/Synonym']),
 (7, ['/c/en/counter/n', '/c/en/person/n', '/r/IsA']),
 (8, ['/c/en/lie/v', '/c/en/with', '/r/RelatedTo']),
 (9, ['/c/en/window/n', '/c/en/car/n', '/r/PartOf']),
 (10, ['/c/en/guy/n', '/c/en/man/n', '/r/IsA']),
 (11, ['/c/en/coat/n', '/c/en/hair/n', '/r/IsA']),
 (12, ['/c/en/face/n', '/c/en/animal/n', '/r/PartOf']),
 (13, ['/c/en/bench/n', '/c/en/seat/n', '/r/IsA']),
 (14, ['/c/en/nose/n', '/c/en/face/n', '/r/PartOf']),
 (15, ['/c/en/window/n', '/c/en/bus/n', '/r/PartOf']),
 (16, ['/c/en/wear/v', '/c/en/have/v', '/r/MannerOf']),
 (17, ['/c/en/mount/v', '/c/en/attach/v', '/r/MannerOf']),
 (18, ['/c/en/bag/n', '/c/en/transport/v', '/r/UsedFor']),
 (19, ['/c/en/bear/n', '/c/en/person/n', '/r/IsA']),
 (20, ['/c/en/cat/n', '/c/en/book/n', '/r/IsA']),
 (21, ['/c/en/roof/n', '/c/en/truck/n', '/r/PartOf']),
 (22, ['/c/en/door/n', '/c/en/room/n', '/r/IsA']),
 (23, ['/c/en/ear/n', '/c/en/head/n', '/r/PartOf']),
 (24, ['/c/en/skateboard/n', '/c/en/board/n', '/r/IsA']),
 (25, ['/c/en/kite/n', '/c/en/bird/n', '/r/IsA']),
 (26, ['/c/en/face/n', '/c/en/head/n', '/r/PartOf']),
 (27, ['/c/en/trunk/n', '/c/en/tree/n', '/r/PartOf']),
 (28, ['/c/en/bowl/n', '/c/en/food/n', '/r/RelatedTo']),
 (29, ['/c/en/chair/n', '/c/en/people/n', '/r/UsedFor']),
 (30, ['/c/en/room/n', '/c/en/building/n', '/r/PartOf']),
 (31, ['/c/en/basket/n', '/c/en/hold/v', '/r/UsedFor']),
 (32, ['/c/en/to', '/c/en/on', '/r/RelatedTo']),
 (33, ['/c/en/flag/n', '/c/en/dog/n', '/r/PartOf']),
 (34, ['/c/en/sit/v', '/c/en/ride/v', '/r/Synonym']),
 (35, ['/c/en/face/n', '/c/en/person/n', '/r/IsA']),
 (36, ['/c/en/paw/n', '/c/en/hand/n', '/r/Synonym']),
 (37, ['/c/en/cover/v', '/c/en/play/v', '/r/MannerOf']),
 (38, ['/c/en/bicycle/n', '/c/en/ride/v', '/r/UsedFor']),
 (39, ['/c/en/with', '/c/en/against', '/r/Antonym']),
 (40, ['/c/en/head/n', '/c/en/animal/n', '/r/PartOf']),
 (41, ['/c/en/bag/n', '/c/en/hold/v', '/r/UsedFor']),
 (42, ['/c/en/pot/n', '/c/en/toilet/n', '/r/Synonym']),
 (43, ['/c/en/boot/n', '/c/en/shoe/n', '/r/IsA']),
 (44, ['/c/en/on', '/c/en/along', '/r/Synonym']),
 (45, ['/c/en/boot/n', '/c/en/car/n', '/r/PartOf']),
 (46, ['/c/en/with', '/c/en/against', '/r/RelatedTo']),
 (47, ['/c/en/hold/v', '/c/en/cover/v', '/r/MannerOf']),
 (48, ['/c/en/counter/n', '/c/en/table/n', '/r/IsA']),
 (49, ['/c/en/motorcycle/n', '/c/en/ride/v', '/r/UsedFor']),
 (50, ['/c/en/leg/n', '/c/en/hold/v', '/r/UsedFor']),
 (51, ['/c/en/horse/n', '/c/en/animal/n', '/r/IsA']),
 (52, ['/c/en/desk/n', '/c/en/table/n', '/r/IsA']),
 (53, ['/c/en/bear/n', '/c/en/animal/n', '/r/IsA']),
 (54, ['/c/en/branch/n', '/c/en/arm/n', '/r/Synonym']),
 (55, ['/c/en/make/v', '/c/en/look/v', '/r/MannerOf']),
 (56, ['/c/en/roof/n', '/c/en/building/n', '/r/PartOf']),
 (57, ['/c/en/flag/n', '/c/en/tail/n', '/r/IsA']),
 (58, ['/c/en/lamp/n', '/c/en/light/n', '/r/RelatedTo']),
 (59, ['/c/en/paint/v', '/c/en/cover/v', '/r/MannerOf']),
 (60, ['/c/en/counter/n', '/c/en/shoe/n', '/r/PartOf']),
 (61, ['/c/en/bench/n', '/c/en/people/n', '/r/RelatedTo']),
 (62, ['/c/en/fly/v', '/c/en/transport/v', '/r/MannerOf']),
 (63, ['/c/en/watch/v', '/c/en/look/v', '/r/Entails']),
 (64, ['/c/en/car/n', '/c/en/transport/v', '/r/UsedFor']),
 (65, ['/c/en/with', '/c/en/on', '/r/RelatedTo']),
 (66, ['/c/en/box/n', '/c/en/player/n', '/r/RelatedTo']),
 (67, ['/c/en/from', '/c/en/to', '/r/DistinctFrom']),
 (68, ['/c/en/leg/n', '/c/en/table/n', '/r/PartOf']),
 (69, ['/c/en/ride/v', '/c/en/lie/v', '/r/MannerOf']),
 (70, ['/c/en/ear/n', '/c/en/fruit/n', '/r/IsA']),
 (71, ['/c/en/player/n', '/c/en/person/n', '/r/IsA']),
 (72, ['/c/en/hand/n', '/c/en/arm/n', '/r/PartOf']),
 (73, ['/c/en/seat/n', '/c/en/airplane/n', '/r/HasContext']),
 (74, ['/c/en/counter/n', '/c/en/hold/v', '/r/UsedFor']),
 (75, ['/c/en/bed/n', '/c/en/hold/v', '/r/UsedFor']),
 (76, ['/c/en/at', '/c/en/on', '/r/RelatedTo']),
 (77, ['/c/en/clock/n', '/c/en/people/n', '/r/UsedFor']),
 (78, ['/c/en/at', '/c/en/to', '/r/RelatedTo']),
 (79, ['/c/en/against', '/c/en/along', '/r/DistinctFrom']),
 (80, ['/c/en/box/n', '/c/en/seat/n', '/r/IsA']),
 (81, ['/c/en/mouth/n', '/c/en/face/n', '/r/PartOf']),
 (82, ['/c/en/hold/v', '/c/en/have/v', '/r/Synonym']),
 (83, ['/c/en/glass/n', '/c/en/hold/v', '/r/UsedFor']),
 (84, ['/c/en/cabinet/n', '/c/en/box/n', '/r/IsA']),
 (85, ['/c/en/boat/n', '/c/en/transport/v', '/r/UsedFor']),
 (86, ['/c/en/from', '/c/en/and', '/r/RelatedTo']),
 (87, ['/c/en/child/n', '/c/en/person/n', '/r/IsA']),
 (88, ['/c/en/walk/v', '/c/en/play/v', '/r/MannerOf']),
 (89, ['/c/en/table/n', '/c/en/board/n', '/r/Synonym']),
 (90, ['/c/en/bus/n', '/c/en/transport/v', '/r/UsedFor']),
 (91, ['/c/en/drawer/n', '/c/en/desk/n', '/r/PartOf']),
 (92, ['/c/en/lamp/n', '/c/en/light/n', '/r/UsedFor']),
 (93, ['/c/en/tile/n', '/c/en/man/n', '/r/IsA']),
 (94, ['/c/en/child/n', '/c/en/animal/n', '/r/IsA']),
 (95, ['/c/en/along', '/c/en/with', '/r/RelatedTo']),
 (96, ['/c/en/leg/n', '/c/en/chair/n', '/r/PartOf']),
 (97, ['/c/en/man/n', '/c/en/person/n', '/r/IsA']),
 (98, ['/c/en/to', '/c/en/at', '/r/RelatedTo']),
 (99, ['/c/en/and', '/c/en/to', '/r/RelatedTo']),
 (100, ['/c/en/with', '/c/en/along', '/r/RelatedTo']),
 (101, ['/c/en/leg/n', '/c/en/branch/n', '/r/Synonym']),
 (102, ['/c/en/cat/n', '/c/en/guy/n', '/r/Synonym']),
 (103, ['/c/en/mouth/n', '/c/en/bottle/n', '/r/PartOf']),
 (104, ['/c/en/make/v', '/c/en/hold/v', '/r/Synonym']),
 (105, ['/c/en/have/v', '/c/en/hold/v', '/r/Synonym']),
 (106, ['/c/en/box/n', '/c/en/hold/v', '/r/UsedFor']),
 (107, ['/c/en/eye/n', '/c/en/face/n', '/r/PartOf']),
 (108, ['/c/en/bus/n', '/c/en/car/n', '/r/IsA']),
 (109, ['/c/en/cup/n', '/c/en/hold/v', '/r/UsedFor']),
 (110, ['/c/en/house/n', '/c/en/building/n', '/r/IsA']),
 (111, ['/c/en/against', '/c/en/with', '/r/Antonym']),
 (112, ['/c/en/make/v', '/c/en/have/v', '/r/Synonym']),
 (113, ['/c/en/gym_shoe/n', '/c/en/shoe/n', '/r/IsA']),
 (114, ['/c/en/toilet/n', '/c/en/room/n', '/r/IsA']),
 (115, ['/c/en/handle/n', '/c/en/racket/n', '/r/PartOf']),
 (116, ['/c/en/shelf/n', '/c/en/cabinet/n', '/r/PartOf']),
 (117, ['/c/en/wave/n', '/c/en/woman/n', '/r/IsA']),
 (118, ['/c/en/against', '/c/en/with', '/r/DistinctFrom']),
 (119, ['/c/en/curtain/n', '/c/en/book/n', '/r/IsA']),
 (120, ['/c/en/make/v', '/c/en/constitute/v', '/r/MannerOf']),
 (121, ['/c/en/jacket/n', '/c/en/coat/n', '/r/IsA']),
 (122, ['/c/en/trunk/n', '/c/en/boot/n', '/r/Synonym']),
 (123, ['/c/en/roof/n', '/c/en/car/n', '/r/PartOf']),
 (124, ['/c/en/counter/n', '/c/en/boot/n', '/r/PartOf']),
 (125, ['/c/en/face/n', '/c/en/racket/n', '/r/PartOf']),
 (126, ['/c/en/stand/n', '/c/en/table/n', '/r/IsA']),
 (127, ['/c/en/chair/n', '/c/en/seat/n', '/r/IsA']),
 (128, ['/c/en/windshield/n', '/c/en/airplane/n', '/r/PartOf']),
 (129, ['/c/en/lady/n', '/c/en/woman/n', '/r/IsA']),
 (130, ['/c/en/finger/n', '/c/en/hand/n', '/r/PartOf']),
 (131, ['/c/en/head/n', '/c/en/toilet/n', '/r/IsA']),
 (132, ['/c/en/wing/n', '/c/en/bird/n', '/r/PartOf']),
 (133, ['/c/en/nose/n', '/c/en/animal/n', '/r/HasContext']),
 (134, ['/c/en/mount/v', '/c/en/ride/v', '/r/Synonym']),
 (135, ['/c/en/curtain/n', '/c/en/window/n', '/r/RelatedTo']),
 (136, ['/c/en/wheel/n', '/c/en/bicycle/n', '/r/Synonym']),
 (137, ['/c/en/surfboard/n', '/c/en/board/n', '/r/IsA']),
 (138, ['/c/en/bed/n', '/c/en/person/n', '/r/RelatedTo']),
 (139, ['/c/en/finger/n', '/c/en/glove/n', '/r/PartOf']),
 (140, ['/c/en/with', '/c/en/from', '/r/RelatedTo']),
 (141, ['/c/en/bed/n', '/c/en/plant/n', '/r/RelatedTo']),
 (142, ['/c/en/handle/n', '/c/en/umbrella/n', '/r/PartOf']),
 (143, ['/c/en/trunk/n', '/c/en/nose/n', '/r/IsA']),
 (144, ['/c/en/window/n', '/c/en/building/n', '/r/PartOf']),
 (145, ['/c/en/roof/n', '/c/en/bus/n', '/r/PartOf']),
 (146, ['/c/en/bottle/n', '/c/en/hold/v', '/r/UsedFor'])]

In [26]:
edges_zareian = [row [1] for row in edges_zareian_enumerate]

In [27]:
list(enumerate(edges_zareian))

[(0, ['/c/en/windshield/n', '/c/en/screen/n', '/r/IsA']),
 (1, ['/c/en/cat/n', '/c/en/woman/n', '/r/IsA']),
 (2, ['/c/en/wing/n', '/c/en/airplane/n', '/r/PartOf']),
 (3, ['/c/en/room/n', '/c/en/to', '/r/RelatedTo']),
 (4, ['/c/en/bowl/n', '/c/en/hold/v', '/r/UsedFor']),
 (5, ['/c/en/play/v', '/c/en/use/v', '/r/MannerOf']),
 (6, ['/c/en/arm/n', '/c/en/branch/n', '/r/Synonym']),
 (7, ['/c/en/counter/n', '/c/en/person/n', '/r/IsA']),
 (8, ['/c/en/lie/v', '/c/en/with', '/r/RelatedTo']),
 (9, ['/c/en/window/n', '/c/en/car/n', '/r/PartOf']),
 (10, ['/c/en/guy/n', '/c/en/man/n', '/r/IsA']),
 (11, ['/c/en/coat/n', '/c/en/hair/n', '/r/IsA']),
 (12, ['/c/en/face/n', '/c/en/animal/n', '/r/PartOf']),
 (13, ['/c/en/bench/n', '/c/en/seat/n', '/r/IsA']),
 (14, ['/c/en/nose/n', '/c/en/face/n', '/r/PartOf']),
 (15, ['/c/en/window/n', '/c/en/bus/n', '/r/PartOf']),
 (16, ['/c/en/wear/v', '/c/en/have/v', '/r/MannerOf']),
 (17, ['/c/en/mount/v', '/c/en/attach/v', '/r/MannerOf']),
 (18, ['/c/en/bag/n', '/c/

In [28]:
edges_zareian[1][1] = '/c/en/animal/n'
edges_zareian[73][2] = '/r/PartOf'
edges_zareian[133][2] = '/r/PartOf'
edges_zareian[136][2] = '/r/PartOf'
edges_zareian[63][2] = '/r/MannerOf'
edges_zareian[22][2] = '/r/PartOf'
edges_zareian[35][2] = '/r/PartOf'
edges_zareian[36][2] = '/r/RelatedTo'
edges_zareian[6][2] = '/r/RelatedTo'
edges_zareian[34][2] = '/r/RelatedTo'
edges_zareian[54][2] = '/r/RelatedTo'
edges_zareian[89][2] = '/r/RelatedTo'
edges_zareian[101][2] = '/r/RelatedTo'
edges_zareian[44][2] = '/r/RelatedTo'
edges_zareian[82][2] = '/r/RelatedTo'
edges_zareian[105][2] = '/r/RelatedTo'
edges_zareian[92][2] = '/r/IsA'
del edges_zareian[141]
del edges_zareian[140]
del edges_zareian[138]
del edges_zareian[134]
del edges_zareian[131]
del edges_zareian[124]
del edges_zareian[122]
del edges_zareian[119]
del edges_zareian[118]
del edges_zareian[117]
del edges_zareian[112]
del edges_zareian[111]
del edges_zareian[104]
del edges_zareian[103]
del edges_zareian[102]
del edges_zareian[99]
del edges_zareian[98]
del edges_zareian[94]
del edges_zareian[93]
del edges_zareian[86]
del edges_zareian[79]
del edges_zareian[78]
del edges_zareian[77]
del edges_zareian[70]
del edges_zareian[67]
del edges_zareian[66]
del edges_zareian[61]
del edges_zareian[60]
del edges_zareian[57]
del edges_zareian[55]
del edges_zareian[45]
del edges_zareian[42]
del edges_zareian[39]
del edges_zareian[37]
del edges_zareian[33]
del edges_zareian[32]
del edges_zareian[29]
del edges_zareian[20]
del edges_zareian[19]
del edges_zareian[11]
del edges_zareian[8]
del edges_zareian[7]
del edges_zareian[3]


In [29]:
edge_types = set([item[2] for item in edges_zareian])
edge_types

{'/r/IsA', '/r/MannerOf', '/r/PartOf', '/r/RelatedTo', '/r/UsedFor'}

In [30]:
edge_freq = {item: 0 for item in edge_types}
for item in edges_zareian:
    edge_freq[item[2]] += 1

In [31]:
symmetric_edge_types = ['/r/Synonym', '/r/RelatedTo']
edge_types_ent2ent = ['/r/RelatedTo', '/r/PartOf', '/r/IsA']
edge_types_pred2pred = ['/r/RelatedTo', '/r/MannerOf']
edge_types_ent2pred = ['/r/UsedFor']

In [32]:
edges_ent2ent = np.zeros((5, 151, 151))
edges_pred2pred = np.zeros((3, 51, 51))
edges_ent2pred = np.zeros((1, 151, 51))
edges_pred2ent = np.zeros((1, 51, 151))
# edges_ent2ent = np.zeros((5, len(label2synset_combined), len(label2synset_combined)))
# edges_pred2pred = np.zeros((3, len(preds_old), len(preds_old)))
# edges_ent2pred = np.zeros((1, len(label2synset_combined), len(preds_old)))
# edges_pred2ent = np.zeros((1, len(preds_old), len(label2synset_combined)))
print(edges_ent2ent.shape)
print(edges_pred2pred.shape)
print(edges_ent2pred.shape)
print(edges_pred2ent.shape)

(5, 151, 151)
(3, 51, 51)
(1, 151, 51)
(1, 51, 151)


In [33]:
id2label = {}
for l, i in label2id.items():
    if i not in id2label:
        id2label[i] = []
    id2label[i].append(l) 
id2pred = {}
for l, i in pred2id.items():
    if i not in id2pred:
        id2pred[i] = []
    id2pred[i].append(l) 
    

In [34]:
for edge in edges_zareian:
    if edge[2] != '/r/IsA':
        continue
    if edge[0] not in id2label or edge[1] not in id2label:
        raise
    for n1 in id2label[edge[0]]:
        for n2 in id2label[edge[1]]:
            edges_ent2ent[0, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['label_to_idx'][n2]] = 1
            edges_ent2ent[1, scene_graph_meta['label_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1
        

In [35]:
for edge in edges_zareian:
    if edge[2] != '/r/PartOf':
        continue
    if edge[0] not in id2label or edge[1] not in id2label:
        raise
    for n1 in id2label[edge[0]]:
        for n2 in id2label[edge[1]]:
            edges_ent2ent[2, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['label_to_idx'][n2]] = 1
            edges_ent2ent[3, scene_graph_meta['label_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1


In [36]:
for edge in edges_zareian:
    if edge[2] != '/r/RelatedTo':
        continue
    if edge[0] not in id2label or edge[1] not in id2label:
        if edge[0] not in id2pred or edge[1] not in id2pred:
            print(edge)
            raise
        continue
    for n1 in id2label[edge[0]]:
        for n2 in id2label[edge[1]]:
            edges_ent2ent[4, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['label_to_idx'][n2]] = 1
            edges_ent2ent[4, scene_graph_meta['label_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1


In [37]:
for edge in edges_zareian:
    if edge[2] != '/r/MannerOf':
        continue
    if edge[0] not in id2pred or edge[1] not in id2pred:
        raise
    for n1 in id2pred[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_pred2pred[0, scene_graph_meta['predicate_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2pred[1, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['predicate_to_idx'][n1]] = 1
        

In [38]:
for edge in edges_zareian:
    if edge[2] != '/r/RelatedTo':
        continue
    if edge[0] not in id2pred or edge[1] not in id2pred:
        if edge[0] not in id2label or edge[1] not in id2label:
            raise
        continue
    for n1 in id2pred[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_pred2pred[2, scene_graph_meta['predicate_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2pred[2, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['predicate_to_idx'][n1]] = 1


In [39]:
for edge in edges_zareian:
    if edge[2] != '/r/UsedFor':
        continue
    if edge[0] not in id2label or edge[1] not in id2pred:
        print(edge)
        continue
    for n1 in id2label[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_ent2pred[0, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2ent[0, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1
        

In [40]:
print(edges_ent2ent.sum(axis=(1,2)))
print(edges_pred2pred.sum(axis=(1,2)))
print(edges_ent2pred.sum(axis=(1,2)))
print(edges_pred2ent.sum(axis=(1,2)))

[30. 30. 37. 37. 14.]
[13. 13. 14.]
[16.]
[16.]


In [41]:
edge_dict_conceptnet = {
    'conceptnet_ent2ent': edges_ent2ent,
    'conceptnet_pred2pred': edges_pred2pred,
    'conceptnet_ent2pred': edges_ent2pred,
    'conceptnet_pred2ent': edges_pred2ent,
}

In [42]:
with open(os.path.join('conceptnet_edges_zareian_code_20211017.pkl'), 'wb') as fout:
    pickle.dump(edge_dict_conceptnet, fout)

In [43]:
{k: v.shape for k, v in edge_dict_conceptnet.items()}

{'conceptnet_ent2ent': (5, 151, 151),
 'conceptnet_pred2pred': (3, 51, 51),
 'conceptnet_ent2pred': (1, 151, 51),
 'conceptnet_pred2ent': (1, 51, 151)}

In [44]:
edge_dict_conceptnet_zareian = edge_dict_conceptnet.copy()

# 3. Add Wikidata Edges without More Nodes?

In [45]:
df_excluded.columns

Index(['entity_head_vgg', 'predicate', 'entity_tail_wikidata', 'synset',
       'cn_id_tail', 'cn_id_head'],
      dtype='object')

In [46]:
edges_wikidata = df_excluded[['cn_id_head', 'cn_id_tail', 'predicate']].reset_index(drop=True).values.tolist()
# df_excluded[['cn_id_head', 'cn_id_tail', 'predicate']].values.tolist()

In [47]:
edges_wikidata

[['/c/en/chair/n', '/c/en/academic_department/n', '/r/IsA'],
 ['/c/en/umbrella/n', '/c/en/accessory/n', '/r/IsA'],
 ['/c/en/helmet/n', '/c/en/accomplishment/n', '/r/PartOf'],
 ['/c/en/handle/n', '/c/en/address/n', '/r/IsA'],
 ['/c/en/tower/n', '/c/en/ride/n', '/r/IsA'],
 ['/c/en/wing/n', '/c/en/structure/n', '/r/IsA'],
 ['/c/en/hair/n', '/c/en/structure/n', '/r/IsA'],
 ['/c/en/bowl/n', '/c/en/artifact/n', '/r/IsA'],
 ['/c/en/cabinet/n', '/c/en/component/n', '/r/IsA'],
 ['/c/en/pole/n', '/c/en/component/n', '/r/IsA'],
 ['/c/en/window/n', '/c/en/component/n', '/r/IsA'],
 ['/c/en/tower/n', '/c/en/structure/n', '/r/IsA'],
 ['/c/en/house/n', '/c/en/structure/n', '/r/IsA'],
 ['/c/en/pole/n', '/c/en/structure/n', '/r/IsA'],
 ['/c/en/roof/n', '/c/en/structure/n', '/r/IsA'],
 ['/c/en/board/n', '/c/en/material/n', '/r/IsA'],
 ['/c/en/post/n', '/c/en/article/n', '/r/IsA'],
 ['/c/en/hand/n', '/c/en/asset/n', '/r/IsA'],
 ['/c/en/house/n', '/c/en/astrology/n', '/r/HasContext'],
 ['/c/en/box/n', '/c/

In [48]:
edges_ent2ent_old = edges_ent2ent.copy()
edges_pred2pred_old = edges_pred2pred.copy()
edges_pred2ent_old = edges_pred2ent.copy()
edges_ent2pred_old = edges_ent2pred.copy()

# Is there any new edge for all the predicates? No

In [49]:
for edge in edges_wikidata:
    if edge[2] == '/r/IsA' and edge[0] in id2label and edge[1] in id2label:
        print(edge)
        
        
for idx, edge in enumerate(edges_wikidata):
    if edge[2] != '/r/IsA':
        continue
    if edge[0] in id2label and edge[1] in id2label:
        print('idx, edge =', idx, edge)
        for n1 in id2label[edge[0]]:
            for n2 in id2label[edge[1]]:
                print('edges_ent2ent[0, {}, {}] was previously'.format(n1, n2), edges_ent2ent[0, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['label_to_idx'][n2]])
                edges_ent2ent[0, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['label_to_idx'][n2]] = 1
                print('edges_ent2ent[1, {}, {}] was previously'.format(n2, n1), edges_ent2ent[1, scene_graph_meta['label_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]])
                edges_ent2ent[1, scene_graph_meta['label_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1
        

['/c/en/branch/n', '/c/en/branch/n', '/r/IsA']
['/c/en/branch/n', '/c/en/branch/n', '/r/IsA']
['/c/en/house/n', '/c/en/building/n', '/r/IsA']
['/c/en/fence/n', '/c/en/building/n', '/r/IsA']
['/c/en/wing/n', '/c/en/building/n', '/r/IsA']
['/c/en/windshield/n', '/c/en/glass/n', '/r/IsA']
['/c/en/coat/n', '/c/en/coat/n', '/r/IsA']
['/c/en/counter/n', '/c/en/man/n', '/r/IsA']
['/c/en/person/n', '/c/en/person/n', '/r/IsA']
['/c/en/lamp/n', '/c/en/light/n', '/r/IsA']
['/c/en/house/n', '/c/en/building/n', '/r/IsA']
['/c/en/post/n', '/c/en/path/n', '/r/IsA']
['/c/en/bench/n', '/c/en/seat/n', '/r/IsA']
['/c/en/flag/n', '/c/en/sign/n', '/r/IsA']
['/c/en/rock/n', '/c/en/rock/n', '/r/IsA']
['/c/en/desk/n', '/c/en/table/n', '/r/IsA']
idx, edge = 40 ['/c/en/branch/n', '/c/en/branch/n', '/r/IsA']
edges_ent2ent[0, branch, branch] was previously 0.0
edges_ent2ent[1, branch, branch] was previously 0.0
idx, edge = 42 ['/c/en/branch/n', '/c/en/branch/n', '/r/IsA']
edges_ent2ent[0, branch, branch] was prev

In [50]:
for edge in edges_zareian:
    if edge[2] != '/r/PartOf':
        continue
    if edge[0] not in id2label or edge[1] not in id2label:
        raise
    for n1 in id2label[edge[0]]:
        for n2 in id2label[edge[1]]:
            edges_ent2ent[2, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['label_to_idx'][n2]] = 1
            edges_ent2ent[3, scene_graph_meta['label_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1


In [51]:
for edge in edges_wikidata:
    if edge[2] == '/r/RelatedTo' and ((edge[0] in id2label and edge[1] in id2label) or (edge[0] in id2pred or edge[1] in id2pred)):
        print(edge)

In [52]:
for idx, edge in enumerate(edges_wikidata):
    if edge[2] != '/r/RelatedTo':
        continue
    if edge[0] not in id2label or edge[1] not in id2label:
        if edge[0] not in id2pred or edge[1] not in id2pred:
            print(edge)
            raise
        continue
    for n1 in id2label[edge[0]]:
        for n2 in id2label[edge[1]]:
            edges_ent2ent[4, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['label_to_idx'][n2]] = 1
            edges_ent2ent[4, scene_graph_meta['label_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1

['/c/en/vehicle/n', '/c/en/vessel/n', '/r/RelatedTo']


RuntimeError: No active exception to reraise

In [ ]:
for edge in edges_wikidata:
    if edge[2] == '/r/MannerOf' and (edge[0] in id2pred and edge[1] in id2pred):
        print(edge)

In [ ]:
for edge in edges_wikidata:
    if edge[2] != '/r/MannerOf':
        continue
    if edge[0] not in id2pred or edge[1] not in id2pred:
        raise
    for n1 in id2pred[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_pred2pred[0, scene_graph_meta['predicate_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2pred[1, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['predicate_to_idx'][n1]] = 1
        

In [ ]:
for edge in edges_wikidata:
    if edge[2] == '/r/RelatedTo' and (edge[0] in id2label and edge[1] in id2label):
        print(edge)

In [ ]:
for edge in edges_wikidata:
    if edge[2] != '/r/RelatedTo':
        continue
    if edge[0] not in id2pred or edge[1] not in id2pred:
        if edge[0] not in id2label or edge[1] not in id2label:
            raise
        continue
    for n1 in id2pred[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_pred2pred[2, scene_graph_meta['predicate_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2pred[2, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['predicate_to_idx'][n1]] = 1


In [ ]:
for edge in edges_wikidata:
    if edge[2] == '/r/UsedFor' and (edge[0] in id2label and edge[1] in id2pred):
        print(edge)

In [ ]:
for edge in edges_wikidata:
    if edge[2] != '/r/UsedFor':
        continue
    if edge[0] not in id2label or edge[1] not in id2pred:
        print(edge)
        continue
    for n1 in id2label[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_ent2pred[0, scene_graph_meta['label_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2ent[0, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['label_to_idx'][n1]] = 1
        

In [ ]:
list(enumerate(edges_zareian))

# 4. Add Wikidata Edges with More Nodes

# TODO: manually prune edges

In [54]:
labels_wikidata_excluded = list(df_excluded['entity_tail_wikidata'])
label2synset_wikidata = {l: s for l, s in zip(labels_wikidata_excluded, wikidata_labels_matched)}

In [58]:
len(label2synset_wikidata)

196

In [55]:
# keep original
label2synset_combined = label2synset_wikidata.copy()
label2synset_combined.update(label2synset)

In [57]:
len(label2synset_combined)

332

In [64]:
edges_ent2ent_plus_wikidata = np.zeros((5, len(label2synset_combined)+1, len(label2synset_combined)+1))
edges_pred2pred_plus_wikidata = np.zeros((3, len(preds)+1, len(preds)+1))
edges_ent2pred_plus_wikidata = np.zeros((1, len(label2synset_combined)+1, len(preds)+1))
edges_pred2ent_plus_wikidata = np.zeros((1, len(preds)+1, len(label2synset_combined)+1))
# print(edges_ent2ent.shape)
# print(edges_pred2pred.shape)
# print(edges_ent2pred.shape)
# print(edges_pred2ent.shape)

In [67]:
dict(sorted(scene_graph_meta['label_to_idx'].items(), key=lambda item: item[1]))

{'airplane': 1,
 'animal': 2,
 'arm': 3,
 'bag': 4,
 'banana': 5,
 'basket': 6,
 'beach': 7,
 'bear': 8,
 'bed': 9,
 'bench': 10,
 'bike': 11,
 'bird': 12,
 'board': 13,
 'boat': 14,
 'book': 15,
 'boot': 16,
 'bottle': 17,
 'bowl': 18,
 'box': 19,
 'boy': 20,
 'branch': 21,
 'building': 22,
 'bus': 23,
 'cabinet': 24,
 'cap': 25,
 'car': 26,
 'cat': 27,
 'chair': 28,
 'child': 29,
 'clock': 30,
 'coat': 31,
 'counter': 32,
 'cow': 33,
 'cup': 34,
 'curtain': 35,
 'desk': 36,
 'dog': 37,
 'door': 38,
 'drawer': 39,
 'ear': 40,
 'elephant': 41,
 'engine': 42,
 'eye': 43,
 'face': 44,
 'fence': 45,
 'finger': 46,
 'flag': 47,
 'flower': 48,
 'food': 49,
 'fork': 50,
 'fruit': 51,
 'giraffe': 52,
 'girl': 53,
 'glass': 54,
 'glove': 55,
 'guy': 56,
 'hair': 57,
 'hand': 58,
 'handle': 59,
 'hat': 60,
 'head': 61,
 'helmet': 62,
 'hill': 63,
 'horse': 64,
 'house': 65,
 'jacket': 66,
 'jean': 67,
 'kid': 68,
 'kite': 69,
 'lady': 70,
 'lamp': 71,
 'laptop': 72,
 'leaf': 73,
 'leg': 74,
 'l

In [71]:
label2order = scene_graph_meta['label_to_idx'].copy()
# append wikidata entities to the end of the alphabetically sorted s
i = len(label2order)+1
for wikidata_label in labels_wikidata_excluded:
    if wikidata_label not in label2order:
        label2order[wikidata_label] = i
        i += 1

In [73]:
len(label2order)

332

In [141]:
# save_obj(label2order, 'label_to_idx_combined_332_20211021')

In [142]:
scene_graph_meta_combined = scene_graph_meta.copy()
scene_graph_meta_combined['label_to_idx'] = label2order

In [143]:
import json
with open('VG-SGG-dicts_combined.json', 'w') as f:
    json.dump(scene_graph_meta_combined, f)

In [74]:
dict(sorted(label2order.items(), key=lambda item: item[1]))

{'airplane': 1,
 'animal': 2,
 'arm': 3,
 'bag': 4,
 'banana': 5,
 'basket': 6,
 'beach': 7,
 'bear': 8,
 'bed': 9,
 'bench': 10,
 'bike': 11,
 'bird': 12,
 'board': 13,
 'boat': 14,
 'book': 15,
 'boot': 16,
 'bottle': 17,
 'bowl': 18,
 'box': 19,
 'boy': 20,
 'branch': 21,
 'building': 22,
 'bus': 23,
 'cabinet': 24,
 'cap': 25,
 'car': 26,
 'cat': 27,
 'chair': 28,
 'child': 29,
 'clock': 30,
 'coat': 31,
 'counter': 32,
 'cow': 33,
 'cup': 34,
 'curtain': 35,
 'desk': 36,
 'dog': 37,
 'door': 38,
 'drawer': 39,
 'ear': 40,
 'elephant': 41,
 'engine': 42,
 'eye': 43,
 'face': 44,
 'fence': 45,
 'finger': 46,
 'flag': 47,
 'flower': 48,
 'food': 49,
 'fork': 50,
 'fruit': 51,
 'giraffe': 52,
 'girl': 53,
 'glass': 54,
 'glove': 55,
 'guy': 56,
 'hair': 57,
 'hand': 58,
 'handle': 59,
 'hat': 60,
 'head': 61,
 'helmet': 62,
 'hill': 63,
 'horse': 64,
 'house': 65,
 'jacket': 66,
 'jean': 67,
 'kid': 68,
 'kite': 69,
 'lady': 70,
 'lamp': 71,
 'laptop': 72,
 'leaf': 73,
 'leg': 74,
 'l

In [76]:
len(edges_zareian)

104

In [79]:
len(id2label)

147

In [83]:
# Output is correct against Zareian's
print(edges_ent2ent_old.shape)
print(edges_pred2pred_old.shape)
print(edges_ent2pred_old.shape)
print(edges_pred2ent_old.shape)

(5, 151, 151)
(3, 51, 51)
(1, 151, 51)
(1, 51, 151)


In [82]:
# Output is correct against Zareian's
print(edges_ent2ent_old.sum(axis=(1,2)))
print(edges_pred2pred_old.sum(axis=(1,2)))
print(edges_ent2pred_old.sum(axis=(1,2)))
print(edges_pred2ent_old.sum(axis=(1,2)))

[30. 30. 37. 37. 14.]
[13. 13. 14.]
[16.]
[16.]


In [84]:
edges_ent2ent_plus_wikidata[:, 0:151, 0:151] = edges_ent2ent_old
edges_pred2pred_plus_wikidata[:, 0:51, 0:51] = edges_pred2pred_old
edges_ent2pred_plus_wikidata[:, 0:151, 0:51] = edges_ent2pred_old
edges_pred2ent_plus_wikidata[:, 0:51, 0:151] = edges_pred2ent_old

In [85]:
# Output is correct against Zareian's
print(edges_ent2ent_old.sum(axis=(1,2)))
print(edges_pred2pred_old.sum(axis=(1,2)))
print(edges_ent2pred_old.sum(axis=(1,2)))
print(edges_pred2ent_old.sum(axis=(1,2)))

[30. 30. 37. 37. 14.]
[13. 13. 14.]
[16.]
[16.]


In [105]:
label2id_wikidata = load_obj('label2id_wikidata_196_20211018')

In [106]:
id2label_wikidata = {}
for l, i in label2id_wikidata.items():
    if i not in id2label_wikidata:
        id2label_wikidata[i] = []
    id2label_wikidata[i].append(l) 

# Important: id2label has to be combined because we check for nodes existence

In [115]:
id2label_combined={ key:id2label.get(key,[])+id2label_wikidata.get(key,[]) for key in set(list(id2label.keys())+list(id2label_wikidata.keys())) }

In [117]:
len(id2label_combined) # 312 because one ConceptNet ID has multiple labels

312

In [111]:
id2label_wikidata

{'/c/en/academic_department/n': ['academic_department'],
 '/c/en/accessory/n': ['accessory'],
 '/c/en/accomplishment/n': ['achievement'],
 '/c/en/address/n': ['address'],
 '/c/en/ride/n': ['amusement_ride'],
 '/c/en/structure/n': ['anatomical_structure', 'architectural_structure'],
 '/c/en/artifact/n': ['archaeological_artifact'],
 '/c/en/component/n': ['architectural_element', 'structural_element'],
 '/c/en/material/n': ['art_material'],
 '/c/en/article/n': ['article'],
 '/c/en/asset/n': ['asset'],
 '/c/en/astrology/n': ['astrology'],
 '/c/en/auditorium/n': ['auditorium'],
 '/c/en/badminton/n': ['badminton'],
 '/c/en/baggage/n': ['baggage'],
 '/c/en/bat/n': ['bat'],
 '/c/en/bedclothes/n': ['bedding'],
 '/c/en/bell_tower/n': ['bell_tower'],
 '/c/en/berlin/n': ['berlin'],
 '/c/en/bin/n': ['bin'],
 '/c/en/blade/n': ['blade'],
 '/c/en/blend/n': ['blend'],
 '/c/en/body_armor/n': ['body_armor'],
 '/c/en/body_part/n': ['body_part'],
 '/c/en/bonnet/n': ['bonnet'],
 '/c/en/box/n': ['box'],
 '/

In [118]:
for edge in edges_wikidata:
    if edge[2] != '/r/IsA':
        continue
    if edge[0] not in id2label_combined or edge[1] not in id2label_combined:
        raise
    for n1 in id2label_combined[edge[0]]:
        for n2 in id2label_combined[edge[1]]:
            edges_ent2ent_plus_wikidata[0, label2order[n1], label2order[n2]] = 1
            edges_ent2ent_plus_wikidata[1, label2order[n2], label2order[n1]] = 1
        

In [119]:
for edge in edges_wikidata:
    if edge[2] != '/r/PartOf':
        continue
    if edge[0] not in id2label_combined or edge[1] not in id2label_combined:
        raise
    for n1 in id2label_combined[edge[0]]:
        for n2 in id2label_combined[edge[1]]:
            edges_ent2ent_plus_wikidata[2, label2order[n1], label2order[n2]] = 1
            edges_ent2ent_plus_wikidata[3, label2order[n2], label2order[n1]] = 1


In [120]:
for edge in edges_wikidata:
    if edge[2] != '/r/RelatedTo':
        continue
    if edge[0] not in id2label_combined or edge[1] not in id2label_combined:
        if edge[0] not in id2pred or edge[1] not in id2pred:
            print(edge)
            raise
        continue
    for n1 in id2label_combined[edge[0]]:
        for n2 in id2label_combined[edge[1]]:
            edges_ent2ent_plus_wikidata[4, label2order[n1], label2order[n2]] = 1
            edges_ent2ent_plus_wikidata[4, label2order[n2], label2order[n1]] = 1


In [121]:
for edge in edges_wikidata:
    if edge[2] != '/r/MannerOf':
        continue
    if edge[0] not in id2pred or edge[1] not in id2pred:
        raise
    for n1 in id2pred[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_pred2pred_plus_wikidata[0, scene_graph_meta['predicate_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2pred_plus_wikidata[1, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['predicate_to_idx'][n1]] = 1
        

In [122]:
for edge in edges_wikidata:
    if edge[2] != '/r/RelatedTo':
        continue
    if edge[0] not in id2pred or edge[1] not in id2pred:
        if edge[0] not in id2label_combined or edge[1] not in id2label_combined:
            raise
        continue
    for n1 in id2pred[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_pred2pred_plus_wikidata[2, scene_graph_meta['predicate_to_idx'][n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2pred_plus_wikidata[2, scene_graph_meta['predicate_to_idx'][n2], scene_graph_meta['predicate_to_idx'][n1]] = 1


In [123]:
for edge in edges_wikidata:
    if edge[2] != '/r/UsedFor':
        continue
    if edge[0] not in id2label_combined or edge[1] not in id2pred:
        print(edge)
        continue
    for n1 in id2label_combined[edge[0]]:
        for n2 in id2pred[edge[1]]:
            edges_ent2pred_plus_wikidata[0, label2order[n1], scene_graph_meta['predicate_to_idx'][n2]] = 1
            edges_pred2ent_plus_wikidata[0, scene_graph_meta['predicate_to_idx'][n2], label2order[n1]] = 1
        

['/c/en/house/n', '/c/en/dwelling/n', '/r/UsedFor']
['/c/en/helmet/n', '/c/en/martial_art/n', '/r/UsedFor']
['/c/en/glove/n', '/c/en/martial_art/n', '/r/UsedFor']
['/c/en/railing/n', '/c/en/protection/n', '/r/UsedFor']
['/c/en/desk/n', '/c/en/reading/n', '/r/UsedFor']
['/c/en/pole/n', '/c/en/sailing/n', '/r/UsedFor']
['/c/en/sign/n', '/c/en/sign_language/n', '/r/UsedFor']
['/c/en/skateboard/n', '/c/en/skateboarding/n', '/r/UsedFor']
['/c/en/ski/n', '/c/en/skiing/n', '/r/UsedFor']
['/c/en/cabinet/n', '/c/en/storage/n', '/r/UsedFor']
['/c/en/pole/n', '/c/en/surveying/n', '/r/UsedFor']
['/c/en/boot/n', '/c/en/torture/n', '/r/UsedFor']
['/c/en/pole/n', '/c/en/pylon/n', '/r/UsedFor']
['/c/en/window/n', '/c/en/ventilation/n', '/r/UsedFor']
['/c/en/bench/n', '/c/en/exercise/n', '/r/UsedFor']


In [125]:
print(edges_ent2ent_plus_wikidata.sum(axis=(1,2)))
print(edges_pred2pred_plus_wikidata.sum(axis=(1,2)))
print(edges_ent2pred_plus_wikidata.sum(axis=(1,2)))
print(edges_pred2ent_plus_wikidata.sum(axis=(1,2)))

[208. 208.  89.  89.  16.]
[13. 13. 14.]
[16.]
[16.]


In [129]:
# edge_dict_conceptnet_plus_wikidata = {
#     'conceptnet_ent2ent': edges_ent2ent_plus_wikidata,
#     'conceptnet_pred2pred': edges_pred2pred_plus_wikidata,
#     'conceptnet_ent2pred': edges_ent2pred_plus_wikidata,
#     'conceptnet_pred2ent': edges_pred2ent_plus_wikidata,
# }

edge_dict_conceptnet_plus_wikidata = {
    'edges_ent2ent': edges_ent2ent_plus_wikidata,
    'edges_pred2pred': edges_pred2pred_plus_wikidata,
    'edges_ent2pred': edges_ent2pred_plus_wikidata,
    'edges_pred2ent': edges_pred2ent_plus_wikidata,
}

In [127]:
print(edges_ent2ent_plus_wikidata.shape)
print(edges_pred2pred_plus_wikidata.shape)
print(edges_ent2pred_plus_wikidata.shape)
print(edges_pred2ent_plus_wikidata.shape)

(5, 333, 333)
(3, 51, 51)
(1, 333, 51)
(1, 51, 333)


In [130]:
with open(os.path.join('edge_dict_conceptnet_plus_wikidata_333_20211018.pkl'), 'wb') as fout:
    pickle.dump(edge_dict_conceptnet_plus_wikidata, fout)

In [134]:
len(edge_dict_conceptnet_plus_wikidata

{'edges_ent2ent': array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]

In [136]:
len(edges_wikidata)

240

In [138]:
len(edges_zareian)

104